# Lab3 - N-puzzle

In [1]:
from tqdm.auto import tqdm
import numpy as np
from icecream import ic
import matplotlib.pyplot as plt
from PIL import Image

This is the configuration.
- change `PUZZLE_DIM` to change the size of the puzzle, considering that it represent the side of the square matrix
- `RANDOMIZE_STEPS` is set to a value that is high enough for tested instances
- other parameter are explained in the `README.md` and can be left like this or tweaked to test different configuration; note that to perform A* it is sufficient to put `HEURISTIC_WEIGHT=1` and `MAX_HEURISTIC_WEIGHT=1`
- note that `HEURISTIC_WEIGHT_MSTEPS` is also used to print some statistics in such steps at runtime

In [2]:
PUZZLE_DIM = 5
RANDOMIZE_STEPS = 100_000

HEURISTIC_WEIGHT = 1.5
HEURISTIC_WEIGHT_MULT=1.05
HEURISTIC_WEIGHT_MSTEPS = 10**PUZZLE_DIM
MAX_HEURISTIC_WEIGHT = 3

Now i provide an easy configuration of the random number generator in order to make the pseudo-randomness controlled and easily reproducible.

In [3]:
rng = np.random.Generator(np.random.PCG64([PUZZLE_DIM, RANDOMIZE_STEPS]))

Now let's define the class `State`, that is used to represent a State in the search space.
The class includes:
- `state`: the actual state as a np array
- `depth`: the current tree depth; if the current is the initial set, this is set to `0`;
- `last_move`: the last move done used to going from parent state to current; if the current is the initial set, this is set to `None`;
- `parent`: the parent state that generated the current state; if the current is the initial set, this is set to `None`;

The class includes a set of methods useful for generation & initialization of state, actions evaluation and execution
and cost computation.

The cost is the sum of the actual cost and the heuristic cost function: in this case, i used **manhattan distance** as heuristic cost function, proven to be the best choice for path searching problems in grid-like environments where movement is restricted to horizontal and vertical directions (**sources**: *colleagues and lesson*, *https://theory.stanford.edu/~amitp/GameProgramming/Heuristics.html*,<br/> *https://pages.cs.wisc.edu/~dyer/cs540/notes/search2.html*)

In [4]:

class State:

    state: np.ndarray
    depth: int
    last_move: str
    parent: "State"

    def __init__(self, state: np.ndarray,parent: "State" = None,last_move:str=None):
        """
        Initialize a new State object.
        Parameters:
            state (np.ndarray): The current state of the puzzle as a NumPy array.
            last_move (str, optional): The last move made to reach the current state. Default is None.
            parent (State, optional): The parent state from which the current state was generated. Default is None.
        """

        self.state = state
        self.parent = parent
        if parent is not None:
            self.depth = parent.depth+1
            if last_move is None:
                raise ValueError("Last move must be specified if parent is given")
            self.last_move = last_move
        else:
            self.depth = 0
            self.last_move = None

    @staticmethod
    def getSolution():
        """
        Generates the goal state for an n-puzzle game as a 2D numpy array with the numbers arranged in ascending order
        from 1 to `PUZZLE_DIM**2 - 1`, with the last position being `0` (representing the empty space).
        """

        return State(np.array([i for i in range(1, PUZZLE_DIM**2)]+[0]).reshape(PUZZLE_DIM, PUZZLE_DIM))

    @staticmethod
    def getRandom():
        """
        Generate a random state for the n-puzzle problem.
        """
        return State(rng.permutation(State.getSolution().state))
    
    def randomize(self, steps: int):
        """
        Randomize the current state by applying `steps` random moves to it.
        """
        for _ in tqdm(range(steps), desc="Randomizing"):
            action = rng.choice(self.available_actions())
            self.state = self.do_action(action,as_child=False).state

    def zeropos(self):
        """
        Get the position of the zero (empty space) in the puzzle.
        """
        for i in range(PUZZLE_DIM):
            for j in range(PUZZLE_DIM):
                if self.state[i,j] == 0:
                    return i,j
        raise ValueError("No zero found")


    def available_actions(self) -> list[str]:
        """
        Get the list of available actions that can be taken in the current state, in format of ["U", "D", "L", "R"],
        considering the action from the point of view of the empty tile (0).
        """
        x, y = self.zeropos()
        actions = list()
        if x > 0:
            actions.append("U") 
        if x < PUZZLE_DIM - 1:
            actions.append("D")
        if y > 0:
            actions.append("L")
        if y < PUZZLE_DIM - 1:
            actions.append("R")
        return actions
    
    def swap(self, pos1, pos2):
        """
        Swap the positions of two elements in the puzzle.
        """
        self.state[pos1], self.state[pos2] = self.state[pos2], self.state[pos1]

    def do_action(self, action: str,*,as_child=True) -> np.ndarray:
        """
        Perform the given action on the current state. If `as_child` is `True`, the action is performed on a child state, otherwise it is performed on the current state.
        Parameters:
            action (str): The action to perform. Must be one of `"U"` (up), `"D"` (down), `"L"` (left), or `"R"` (right).
            as_child (bool, optional): If `True`, create a new state as a child of the current state, and applies the action to it. If `False`, apply the action directly to the current state. Default is `True`.
        Returns:
            State: The state after applying the action.
        """

        if as_child:
            new_state = State(self.state.copy(),self,action)
        else:
            new_state = self # directly apply the action to the current state
        
        x, y = new_state.zeropos()
        if action == "U":
            new_state.swap((x, y), (x-1, y))
        elif action == "D":
            new_state.swap((x, y), (x+1, y))
        elif action == "L":
            new_state.swap((x, y), (x, y-1))
        elif action == "R":
            new_state.swap((x, y), (x, y+1))
        else:
            raise ValueError("Invalid action")
        return new_state
    
    def cost(self):
        """
        Actual cost of the state in terms of the number of moves made to reach it.
        """
        return self.depth
    
    def manhattan_distance(self):
        """
        Calculate the Manhattan distance of the current state from the goal state.
        """
        distance = 0
        for i in range(PUZZLE_DIM):
            for j in range(PUZZLE_DIM):
                if self.state[i, j] == 0:
                    continue # skip the hole
                d, m = divmod(self.state[i, j] - 1, PUZZLE_DIM)
                distance += abs(d - i) + abs(m - j)
        return distance
    
    def tot_cost(self):
        """
        Compute the total cost of the state, which is the sum of the actual cost and the heuristic cost, that in this case is the Manhattan distance.
        Moreover, the heuristic cost is weighted by the factor `HEURISTIC_WEIGHT`.
        Changing the value of `HEURISTIC_WEIGHT` will change the importance of the heuristic cost in the A* algorithm,
        speeding up the search but potentially leading to suboptimal solutions.
        """
        return self.cost() + HEURISTIC_WEIGHT*self.manhattan_distance()
    
    def is_solved(self):
        """
        Check if the current state is the goal state.
        """
        return np.array_equal(self.state, State.getSolution().state)

    def __str__(self):
        return str(self.state)
    
    def __repr__(self):
        return str(self.state)
    
    def __eq__(self, other):
        return np.array_equal(self.state, other.state)
    
    def __hash__(self):
        return hash(self.state.tobytes())
    
    def __lt__(self, other):
        # necessary for priority queue, as it uses the < operator in each tuple item
        return self.tot_cost() < other.tot_cost()
    

Here i also designed some utility functions in order to reconstruct the path and to print the solution in different formats, including a gif visualization executed by `save_seq_as_gif`.
Regarding format, note that the multitute of function is because of the different point of views that actions can be viewed from, if considering the moving hole or its neighbor tiles.

In [5]:
def reconstruct_path(state: State):
    """ Reconstructs the path from the initial state to the given state from the hole POV """
    path = list()
    if state is not None:
        while state.last_move is not None:
            path.append(state.last_move)
            state = state.parent
    return path[::-1] # reverse the path

def hole_to_tile(haction:str):
    """ Returns the opposite encoding, switching from moving the hole to moving the tile """
    if haction == "U":
        return "D"
    if haction == "D":
        return "U"
    if haction == "L":
        return "R"
    if haction == "R":
        return "L"
    raise ValueError("Invalid action")

def expand_encoding(act:str)->str:
    """ Returns the full action string """
    if act == "U":
        return "Up"
    if act == "D":
        return "Down"
    if act == "L":
        return "Left"
    if act == "R":
        return "Right"
    raise ValueError("Invalid action")

def movingTilePos(parent_state:State,tile_action:str):
    """ Returns the position of the tile before the movement """
    x,y = parent_state.zeropos()
    if tile_action == "U":
        return x+1,y
    if tile_action == "D":
        return x-1,y
    if tile_action == "L":
        return x,y+1
    if tile_action == "R":
        return x,y-1
    raise ValueError("Invalid action")

def explain_path(state: State):
    """ Explains the path printing the actions the user would do (in tile encoding) """
    path = list()
    if state is not None:
        while state.last_move is not None:
            tile_action = hole_to_tile(state.last_move)
            state = state.parent
            tpos = movingTilePos(state,tile_action)
            path.append((tile_action,tpos))
    path = path[::-1] # reverse the path
    for action, pos in path:
        print(f"Move tile \"{state.state[pos]}\" {expand_encoding(action)}")

def save_seq_as_gif(state:State, output_filename:str,*, frame_duration:int=500, end_pause_duration:int=2000):
    """ Visualizes the path in a gif; gifs can be parametrized with proper kwargs """
    states = list()
    while state is not None:
        states.append(state.state)
        state = state.parent
    states = states[::-1]

    images = []
    fig, ax = plt.subplots(figsize=(PUZZLE_DIM, PUZZLE_DIM))
    ax.axis("off")
    for st in states:
        ax.clear()

        ax.set_xticks(np.arange(-0.5, PUZZLE_DIM, 1), minor=False)
        ax.set_yticks(np.arange(-0.5, PUZZLE_DIM, 1), minor=False)
        ax.grid(color="black", linestyle="-", linewidth=2)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.invert_yaxis() # Invert the y-axis to make rows appear from top to bottom

        for i in range(PUZZLE_DIM):
            for j in range(PUZZLE_DIM):
                if st[i, j] == 0:
                    ax.add_patch(plt.Rectangle((j-0.5, i-0.5), 1, 1, color="black"))
                else:
                    ax.text(j, i, str(st[i, j]), ha="center", va="center", fontsize=20)

        fig.canvas.draw()
        img = np.frombuffer(fig.canvas.buffer_rgba(), dtype="uint8")
        img = img.reshape(fig.canvas.get_width_height()[::-1] + (4,))
        img = img[:, :, :3]  # Extract only RGB channels if required
        images.append(Image.fromarray(img))
    plt.close(fig)
    
    # add pause on last frame
    end_pause_frames = int(end_pause_duration // frame_duration)
    images.extend([images[-1]] * end_pause_frames)
    
    # save the gif
    images[0].save(
        output_filename,
        save_all=True,
        append_images=images[1:],
        duration=frame_duration,
        loop=0
    )

Let's try to create a randomized solution:

In [6]:

state = State.getSolution()
state.randomize(RANDOMIZE_STEPS)
ic(state)
ic(state.manhattan_distance());

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

ic| state: [[23 22 21  8  3]
            [19  0 18 13 15]
            [ 6 11 12  9  1]
            [24  4 17 20 14]
            [10  5 16  7  2]]
ic| state.manhattan_distance(): np.int64(82)


## A* Algorithm

Finally, let's design the core of the __A* algorothm__.

As first step, let's define the class `InvPriorityQueue` to implement an inverse priority queue using `heapq` package internally.
Also the `StatePriorityQueue` is designed as inherited class of the former specifically designed to append states using as inverse priority their total cost.

In [7]:
import heapq

class InvPriorityQueue:
    def __init__(self):
        self.elements = []
    
    def empty(self):
        return len(self.elements) == 0
    
    def append(self, item, inv_priority):
        heapq.heappush(self.elements, (inv_priority, item))
    
    def popleft(self):
        return heapq.heappop(self.elements)[1]
    
    def __bool__(self):
        """ Override the boolean operator to allow direct check with while and if statements """
        return not self.empty()
    
    def __len__(self):
        return len(self.elements)
    
    def __str__(self):
        return str(self.elements)
    
    def __repr__(self):
        return str(self.elements)
    
class StatePriorityQueue(InvPriorityQueue):
    def append(self, item: State):
        super().append(item, item.tot_cost())

Here we can therefore generate the initial condition of the problem, using the previously defined variable `state`:

In [8]:
frontier = StatePriorityQueue()
frontier.append(state)
visited = set()
visited.add(state)
ic(frontier.elements);

ic| frontier.elements: [(np.float64(123.0),
                         [[23 22 21  8  3]
                        [19  0 18 13 15]
                        [ 6 11 12  9  1]
                        [24  4 17 20 14]
                        [10  5 16  7  2]])]


Finally, we can run the core of the A* algorithm. The main loop is small, but it is enlarged by the added condition for the increase of heuristic weight and statistic print every `HEURISTIC_WEIGHT_MSTEPS` cycles.

In [9]:
cycle=0
while frontier:
    current_state = frontier.popleft()
    visited.add(current_state)
    if current_state.is_solved():
        print("Solved!")
        break
    actions = current_state.available_actions()
    for action in actions:
        new_state = current_state.do_action(action)
        if new_state not in visited:
            frontier.append(new_state)
    """
    - changes in the heuristic weight and print the current state and statistics every HEURISTIC_WEIGHT_MSTEPS cycles
    - printing statistics
    """
    if cycle % HEURISTIC_WEIGHT_MSTEPS == 0:
        print(f"current_state (@cycle-{cycle}):\n{current_state}")
        print(f"current statistics:\nevalued states: {len(visited)}\nfrontier states: {len(frontier)}")
        if HEURISTIC_WEIGHT < MAX_HEURISTIC_WEIGHT:
            old_heuristic_weight = HEURISTIC_WEIGHT
            HEURISTIC_WEIGHT *= HEURISTIC_WEIGHT_MULT
            if HEURISTIC_WEIGHT > MAX_HEURISTIC_WEIGHT:
                HEURISTIC_WEIGHT = MAX_HEURISTIC_WEIGHT
            print(f"incrementing heuristic weight from {old_heuristic_weight} to {HEURISTIC_WEIGHT}\n")
        else:
            print(f"heuristic weight is already at max value: {HEURISTIC_WEIGHT}\n")
    cycle += 1

current_state (@cycle-0):
[[23 22 21  8  3]
 [19  0 18 13 15]
 [ 6 11 12  9  1]
 [24  4 17 20 14]
 [10  5 16  7  2]]
current statistics:
evalued states: 1
frontier states: 4
incrementing heuristic weight from 1.5 to 1.5750000000000002

current_state (@cycle-100000):
[[23 21  8  3 15]
 [ 6 22 13  9  1]
 [11 12 19  4 14]
 [16 10 17  5 20]
 [ 0 24  7 18  2]]
current statistics:
evalued states: 99546
frontier states: 110699
incrementing heuristic weight from 1.5750000000000002 to 1.6537500000000003

current_state (@cycle-200000):
[[ 6 23  8  1  3]
 [21  0 22  9  5]
 [11 12 13 14 15]
 [10  7 19  4 20]
 [16 17 18 24  2]]
current statistics:
evalued states: 197337
frontier states: 250375
incrementing heuristic weight from 1.6537500000000003 to 1.7364375000000003

current_state (@cycle-300000):
[[23 22 21  8  3]
 [ 6 12 19  9 15]
 [11 13  1  5 14]
 [16 10 18  2  4]
 [24  7 17  0 20]]
current statistics:
evalued states: 295682
frontier states: 371888
incrementing heuristic weight from 1.7364375

In [10]:
def hprint(*args,**kwargs):
    """Equivalent to print, but highlighted."""
    print('\x1b[1;30;42m' + args[0] + '\x1b[0m',*args[1:],**kwargs)

Check the solution:

In [11]:
print("goal has been reached:")
print(current_state)
hprint(f"QUALITY: SOLUTION consists of {current_state.cost()} actions")
hprint(f"COST: THE NUMBER OF EVALUATED STATES IS {len(visited)}, SO THE NUMBER OF EVALUATED ACTIONS IS {len(visited)-1}")
print("This is the path to be followed by a user to reach the goal from the initial state:")
explain_path(current_state)

goal has been reached:
[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23 24  0]]
QUALITY: SOLUTION consists of 156 actions
COST: THE NUMBER OF EVALUATED STATES IS 2133252, SO THE NUMBER OF EVALUATED ACTIONS IS 2133251
This is the path to be followed by a user to reach the goal from the initial state:
Move tile "19" Right
Move tile "6" Up
Move tile "11" Left
Move tile "12" Left
Move tile "18" Down
Move tile "13" Left
Move tile "9" Up
Move tile "1" Left
Move tile "14" Up
Move tile "20" Right
Move tile "17" Right
Move tile "4" Right
Move tile "5" Up
Move tile "16" Left
Move tile "7" Left
Move tile "20" Down
Move tile "17" Right
Move tile "4" Right
Move tile "24" Right
Move tile "10" Up
Move tile "5" Left
Move tile "16" Left
Move tile "7" Left
Move tile "20" Down
Move tile "17" Right
Move tile "4" Right
Move tile "5" Up
Move tile "16" Left
Move tile "17" Down
Move tile "12" Down
Move tile "18" Down
Move tile "0" Right
Move tile "11" Up
Move tile "12" Left
M

Finally, run the following cell to output the solution as a gif in `.gifs/`

In [12]:
if PUZZLE_DIM <= 3:
    frame_duration = 500
    end_pause_duration = 2000
else:
    frame_duration = 500/(2**(PUZZLE_DIM-3))
    end_pause_duration = 2000*(2**(PUZZLE_DIM-3))
save_seq_as_gif(current_state,f"./gifs/{PUZZLE_DIM}x{PUZZLE_DIM}-puzzle_solution.gif",frame_duration=frame_duration, end_pause_duration=end_pause_duration);